##### Compare our code with Dr. Adams solution

In [1]:
# import pickle file and print the content
import pickle

# variableAndQmatrix.pickle
with open('variableAndQmatrix.pickle', 'rb') as f:
    data = pickle.load(f)


In [2]:
import numpy as np
# get the determinant of each Q matrix
for variable, Qmatrix in data.items():
    print(variable, sep='\n')
    print('Determinant:', (np.linalg.det(Qmatrix), 2))


EP_DISABL
Determinant: (array([0.]), 2)
EP_PCI
Determinant: (array([0.]), 2)
EP_LIMENG
Determinant: (array([-1.17615891e-10]), 2)
EP_CROWD
Determinant: (array([0.]), 2)
EP_UNINSUR
Determinant: (array([3.91085014e-15]), 2)


In [14]:
Qmatrix = data['EP_DISABL'][0]

In [15]:
# ger the shape of the df
print(Qmatrix.shape)

(11, 11)


##### Dr. Adams code

In [16]:
import scipy as sp
from scipy.linalg import solve
from scipy.sparse.linalg import spsolve

In [17]:
# Qmatrix dataframe to numpy array
Qmatrix_ = Qmatrix.to_numpy()

In [18]:
n = Qmatrix_.shape[0]
n

11

In [19]:
# Add jitter to the diagonal of 
# In the context of the provided code, "jittered" refers to the process of adding 
# small random variations to the diagonal elements of the matrix Qmatrix_. 
# This is achieved by multiplying a sparse diagonal matrix (constructed with ones along the main diagonal)
# by the maximum value along the diagonal of Qmatrix_ and the square root of the machine epsilon.

Q_jitter = Qmatrix_ + sp.sparse.diags(np.ones(n)) * max(Qmatrix_.diagonal()) * np.sqrt(

    np.finfo(np.float64).eps

)

In [42]:
# inverse of precision (Q) is cov

# Convert to sparse CSC matrix
Q_perturbed = sp.sparse.csc_array(Q_jitter)

# The CSC (Compressed Sparse Column) format is a way of representing sparse matrices in a compressed form, 
# particularly efficient for matrices with many zeros. In CSC format, the matrix is stored as three one-dimensional arrays:
# one for the non-zero values, one for the row indices, and one for the column indices.

# The spsolve function is used to solve the linear system Q_perturbed * sigma = b for sigma, where
b = sp.sparse.identity(n, format='csc')

# sigma is the inverse of Q_perturbed
sigma = spsolve(Q_perturbed, b)

# If matrices A and B satisfy the equation A⋅B=I, 
# where I is the identity matrix, then B is the inverse of A, and A is the inverse of B.

# this inverse of Q_perturbed is the covariance matrix of the Q_perturbed in bayesian statistics

In [50]:
# V \in Null(Q)

# defines V rather than solving for it from V.T b=0
# In R they have nrow=1, ncol=12 as the shape of V. but on here it is ncol=1, nrow=12 
# becase of that we use V.T instead of V
V = np.ones(n)  # from pg. 6

W = sigma @ V.T  # \Sigma * B in 3.17

# 
Q_inv = sigma - np.outer(W * solve(V @ W, np.ones(1)), W.T)

In [45]:
V.shape

(11,)

In [46]:
W.shape

(11,)

In [47]:
V.dot(W)

246065834.2222774

In [44]:
np.ones(1)

array([1.])

In [36]:
V.shape

(11,)

In [37]:
W.shape

(11,)

In [51]:
# grabbing diag of cov gives var and

# arithmetic mean in log-space becomes geometric mean after exp

scaling = np.exp(np.sum(np.log(np.diag(Q_inv))) / n)

print(scaling)

1302099.5075113948
